Моя программа генератора текста состоит из двух частей: 
1) обучение модели и сохранение получившегося словаря в заданный файл,
2) генерация текста на основе ранее созданной и сохраненной базы, с использованием N-грамной модели.

Функция read_data_from_folder() считывает тексты формата .txt из заданной папочки и возвращает их.  

In [ ]:
import os
import pickle
import random


def read_data_from_folder(folder_path):
    all_text = ''
    for file in os.listdir(path=folder_path):
        if file.endswith('.txt'):
            with open(file, encoding='ANSI') as f:
                my_text = f.read()
            all_text += my_text
    return all_text

Функция make_dict(data) создает словарь биграмов. Ключами будут префиксы, значениями - словари, в которых ключи -- всевозможные
постфиксы после заданного префикса, значения -- кол-во одинаковых постфиксов после данного префикса.
Для слов в конце предложения ('.', '?', '!', '...') постфикс -- 'Ending'. Для слов в начале предложения (следуют после '.', '?', '!', '...') префикс -- 'Beginning'.

In [ ]:
def make_dict(data):  
    dict_of_words = {} 
    words = data.split(' ')
    index = 2    # Первоначальное значение index -- это N из определения N-грамов.

    dict_of_words['Beginning'] = {}    # К ключу 'Beginning' добавляем самое первое слово из всех текстов.
    dict_of_words['Beginning'].update({words[0]: 1})
 
    for word in words[2:]:
        word_low = word.lower()
        key = ' '.join(words[index - 1: index]).lower()
        if key in dict_of_words:
            if key.endswith('.') or key.endswith('!') or key.endswith('?') or key.endswith('...'):
                if 'Ending' in dict_of_words[key]:
                    dict_of_words[key]['Ending'] += 1
                else:
                    dict_of_words[key].update({'Ending': 1})

                if word in dict_of_words['Beginning']:
                    dict_of_words['Beginning'][word] += 1
                else:
                    dict_of_words['Beginning'].update({word: 1})
            else:
                if word_low in dict_of_words[key]:
                    dict_of_words[key][word_low] += 1
                else:
                    dict_of_words[key].update({word_low: 1})
        else:
            if key.endswith('.') or key.endswith('!') or key.endswith('?') or key.endswith('...'):
                dict_of_words[key] = {}
                dict_of_words[key].update({'Ending': 1})
            else:
                dict_of_words[key] = {}
                dict_of_words[key].update({word_low: 1})
        index += 1

    return dict_of_words

Дальше функция download_dict(my_dict, file_name) позволяет сохранить словарь биграмов в заданный файл.

In [ ]:
def download_dict(my_dict, file_name):        
    with open(file_name, 'wb') as out:
        pickle.dump(my_dict, out)

Переходим ко второй части программы -- генерации текста с применением базы, полученной в первой части программы.

Сначала функция dictionary(file_name) загружает сохраненный словарь биграмов из файла.

In [ ]:
def dictionary(file_name):
    with open(file_name, 'rb') as inp:
        loaded_dict = pickle.load(inp)
    return loaded_dict

Затем функция make_text(my_dict, text_length) генерирует текст заданной длины на основе данного ей словаря биграмов. 
Для генерации первого слова находим в словаре ключ 'Beginning', проходимся по словарю, который лежит в его значении, и вытаскиваем отдельно постфиксы и отдельно соответствующие им кол-ва повторений. Самое первое слово в генерируемом тексте -- это рандомный выбор (с весами!) из списка постфиксов к префиксу 'Beginning'.
Далее в цикле идет генерация остальных слов: берем префикс, рандомно с весами выбираем постфикс, двигаемся дальше... (ура, цепи Маркова пригодились)
Важный момент! Если на какой-то префикс рандомайзер выдал 'Ending', то мы этот 'Ending' в текст не включаем, а ищем постфикс для 'Beginning'.

Если вдруг (в моей программе такого получаться не должно, но перестрахуемся) в словаре не был найден ключ, то функция просто возвращает нам все то, что успело сгенерироваться до этого момента.

In [ ]:
def make_text(my_dict, text_length):
    my_values = []
    my_weights = []
    for k, v in my_dict['Beginning'].items():
        my_values.append(k)
        my_weights.append(v)
    first_word = random.choices(my_values, weights=my_weights) 
    text = ''.join(first_word) + ' '
    prefix = ''.join(first_word)

    for i in range(text_length):
        try:
            my_values = []
            my_weights = []
            if prefix == 'Beginning':
                for k, v in my_dict['Beginning'].items():
                    my_values.append(k)
                    my_weights.append(v)
            else:
                for k, v in my_dict[prefix.lower()].items():
                    my_values.append(k)
                    my_weights.append(v)
            new_word = ''.join(random.choices(my_values, weights=my_weights))
            if new_word == 'Ending':
                prefix = 'Beginning'
            else:
                text += new_word + ' '
                prefix = new_word

        except KeyError:
            return text
        
    return text

Все, что осталось, -- запустить функции с нужными аргументами и наслаждаться. :)

In [ ]:
my_dictionary = make_dict(read_data_from_folder('C:\Александра\Прога\генератор текстов\тексты'))
download_dict(my_dictionary, 'C:\Александра\Прога\генератор текстов\dictionary.txt')
print(make_text(dictionary('C:\Александра\Прога\генератор текстов\dictionary.txt'), 1000))

Тексты, использованные для обучения модели:
Д.Дэфо "Робинзон Крузо"
Дойль Артур Конан "Шерлок Холмс"
М.Митчелл "Унесенный ветром"
Э.Юдковски "Гарри Поттер и методы рационального мышления"
Аристотель "Метафизика"
Дюма "Три мушкетера"
Гиляровский "Москва и Москвичи"
Радищев "Путешествие из Петербурга в Москву"
Л.Н.Толстой "Анна Каренина"
Дж.Лукас "Звездные войны"
Дж.Остин "Гордость и предубеждение"
А.П.Чехов Рассказы
Гегель "Наука логики"
Тургенев "Отцы и дети"
К.Льюис "Хроники Нарнии"
Р.Фейнман "Вы, конечно, шутите, мистер Фейнман"